In [1]:
!wget https://huggingface.co/NOCAIR/phyto_class_ucsc_updated/blob/main/phyto_class_ucsc_patched.h5
!wget https://huggingface.co/NOCAIR/phyto_class_ucsc_updated/blob/main/phyto_class_ucsc_class_list.json

--2025-03-31 14:28:16--  https://huggingface.co/NOCAIR/phyto_class_ucsc_updated/blob/main/phyto_class_ucsc_patched.h5
Resolving huggingface.co (huggingface.co)... 52.84.90.106, 52.84.90.122, 52.84.90.129, ...
Connecting to huggingface.co (huggingface.co)|52.84.90.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54868 (54K) [text/html]
Saving to: ‘phyto_class_ucsc_patched.h5’

phyto_class_ucsc_pa 100%[===================>]  53.58K  --.-KB/s    in 0.003s  

2025-03-31 14:28:16 (20.2 MB/s) - ‘phyto_class_ucsc_patched.h5’ saved [54868/54868]

--2025-03-31 14:28:17--  https://huggingface.co/NOCAIR/phyto_class_ucsc_updated/blob/main/phyto_class_ucsc_class_list.json
Resolving huggingface.co (huggingface.co)... 52.84.90.122, 52.84.90.106, 52.84.90.33, ...
Connecting to huggingface.co (huggingface.co)|52.84.90.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93105 (91K) [text/html]
Saving to: ‘phyto_class_ucsc_class_list.json’

phyto_cla

In [9]:
!pip install tensorflow
!pip install keras_preprocessing

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import keras_preprocessing.image as keras_img
import numpy as np
import json
import pandas as pd
from IPython.display import clear_output
import mlcroissant as mlc
import keras

model = keras.models.load_model("./phyto_class_ucsc_patched.h5")
with open("./phyto_class_ucsc_class_list.json") as json_file:
    class_list = list(json.load(json_file))

ds = mlc.Dataset("http://crab.noc.soton.ac.uk/api/v1/exports/af661f0b-e510-4b0b-b97c-2af83190bd42/croissant")
metadata = ds.metadata.to_json()
clear_output(wait=True)
print(f"{metadata['name']}: {metadata['description']}")

records = []
for i in ds.records(record_set="default"):
    records.append(i)

img_stack = np.empty(shape=(len(records),224,224,3))
manual_names = []

def prediction_dataframe(yhat):
    top_ix = np.argmax(yhat,axis=1)
    top_prob = []
    top_class = []
    for i, ix in enumerate(top_ix):
        top_prob.append(yhat[i,ix])
        top_class.append(class_list[ix])

    return pd.DataFrame(data= {"manually_annotated":manual_names, "top_class":top_class,"top_prob":top_prob})
    
x=0
for i in records:
    target_size=(224,224)
    img = keras_img.img_to_array(i["image_content"].resize(target_size))
    img /= 255
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img_stack[x,:,:,:] = img
    manual_names.append(i["image_category"].decode("utf-8"))
    if x > 8000:
        break;
    x += 1

yhat = model.predict(img_stack)
df = prediction_dataframe(yhat)

pd.set_option("display.max_rows", None, "display.max_columns", None)
display(df)

ifcb_test_2025-03-10: IFCB test project!


12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 250ms/step


,manually_annotated,top_class,top_prob
0,Dictyocha,Dictyocha,0.997212
1,Strombidium,Ciliates,0.995359
2,Strombidium,Dinophysis,0.341890
3,Dictyocha,Dictyocha,0.890595
4,Dictyocha,NanoP_less10,0.829418
5,Thalassionema,Asterionellopsis,0.973733
6,Strombidium,Ciliates,0.975499
7,Pleurosigma,Pleurosigma,0.808873
8,Pleurosigma,Pleurosigma,0.999998
9,Thalassionema,Asterionellopsis,0.937588
